In [1]:
from pyfiles.scrpt2_data_transformation import *



Empirical Distribution of Default 90 DPD
      default_type90  count       pct
0         nondefault    729  0.825595
1  default w/in 2yrs     81  0.091733
2   default w/in 1yr     43  0.048698
3  default w/in 3yrs     25  0.028313
4  default w/in 4yrs      5  0.005663


Default Definition Counts
PD90_1yr
nondefault    840
default        43
Name: count, dtype: int64
PD90_2yr
nondefault    759
default       124
Name: count, dtype: int64
Nesting violations: 0


/Users/klemanroy/Downloads/stratus/v2/modeling_v3/script/pyfiles/scrpt2_data_transformation.py:410: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  map_tmoinfo["original_column"].astype(str).str.contains(pattern, na=False),


In [3]:

# Log + z-score blocks (these columns are NOT in your provided mapping; keep original names)
log_cols = [
    "brrtc", "brrta", "brrtt", "brrtb",
    "cbrab", "cbrrb", "cbrrt", "cbrrc"
    ]

for c in log_cols:
    if c in df_loantape_anly.columns:
        df_loantape_anly[f"{c}Ln"] = np.log(num(df_loantape_anly[c]) + 1)

z_cols = [
    "brrgy", "orgtt", "brrfc", "brrtc", "brrta", "brrdt", "brrtt", "brrtb", 
    "cbrau_age", "cbrra", "cbrab", "cbrrb", "cbrrd", "cbrrt", "cbrrc",
]
for c in z_cols:
    if c in df_loantape_anly.columns:
        df_loantape_anly[f"{c}_z"] = zscore(df_loantape_anly[c])

# Citizenship NA-fill rules (these are also not in your provided mapping; keep original names)
def _fill_Y_implies_other_N(df, y_col, other_cols):
    for oc in other_cols:
        if y_col in df.columns and oc in df.columns:
            df[oc] = np.where((df[y_col] == "Y") & (df[oc].isna()), "N", df[oc])
    return df

df_loantape_anly = _fill_Y_implies_other_N(df_loantape_anly, "ctzna", ["usrsd", "vsnla"])
df_loantape_anly = _fill_Y_implies_other_N(df_loantape_anly, "usrsd", ["ctzna", "vsnla"])
df_loantape_anly = _fill_Y_implies_other_N(df_loantape_anly, "vsnla", ["ctzna", "usrsd"])

# This can be replaced by the column called "cbrry" = co_borrower_1_citizenship 
# df_loantape_anly = _fill_Y_implies_other_N(df_loantape_anly, "coborrower_us_citizen", ["coborrower_uspres", "coborrower_visa_only"])
# df_loantape_anly = _fill_Y_implies_other_N(df_loantape_anly, "coborrower_uspres", ["coborrower_us_citizen", "coborrower_visa_only"])
# df_loantape_anly = _fill_Y_implies_other_N(df_loantape_anly, "coborrower_visa_only", ["coborrower_us_citizen", "coborrower_uspres"])

# Interaction vars (original names)
df_loantape_anly["borrower_fico_dti_interxn"] = num(df_loantape_anly.get("brrfc")) * num(df_loantape_anly.get("brrdt"))
df_loantape_anly["borrower_fico_tliab_interxn"] = num(df_loantape_anly.get("brrfc")) * num(df_loantape_anly.get("brrtb"))
df_loantape_anly["borrower_fico_lasset_interxn"] = num(df_loantape_anly.get("brrfc")) * num(df_loantape_anly.get("brrtc"))

df_loantape_anly["borrower_tliab_to_tincome"] = safe_div(df_loantape_anly.get("brrtb"), df_loantape_anly.get("brrtt"))
df_loantape_anly["borrower_tliab_to_tasset"] = safe_div(df_loantape_anly.get("brrtb"), df_loantape_anly.get("brrta"))
df_loantape_anly["borrower_lasset_to_tasset"] = safe_div(df_loantape_anly.get("brrtc"), df_loantape_anly.get("brrta"))
df_loantape_anly["borrower_dti_to_liab"] = safe_div(df_loantape_anly.get("brrdt"), df_loantape_anly.get("brrtb"))

df_loantape_anly["borrower_tliab_to_tassetsLn"] = np.log(num(df_loantape_anly.get("brrtb")) + 1) - np.log(num(df_loantape_anly.get("brrta")) + 1)
df_loantape_anly["borrower_tincome_to_tassetLn"] = np.log(num(df_loantape_anly.get("brrtt")) + 1) - np.log(num(df_loantape_anly.get("brrta")) + 1)
df_loantape_anly["borrower_tliab_to_tincomeLn"] = np.log(num(df_loantape_anly.get("brrtb")) + 1) - np.log(num(df_loantape_anly.get("brrtt")) + 1)

inter_cols = [
    "borrower_fico_dti_interxn",
    "borrower_fico_tliab_interxn",
    "borrower_fico_lasset_interxn",
    "borrower_fico_netincome_interxn",
    "borrower_tliab_to_tincome",
    "borrower_tliab_to_tasset",
    "borrower_netincome_to_disincome",
    "borrower_lasset_to_tasset",
    "borrower_disincome_to_tincome",
    "borrower_dti_to_liab",
    "borrower_tliab_to_tassetsLn",
    "borrower_tincome_to_tassetLn",
    "borrower_tliab_to_tincomeLn",
]
for c in inter_cols:
    if c in df_loantape_anly.columns:
        df_loantape_anly[f"{c}_z"] = zscore(df_loantape_anly[c])


/Users/klemanroy/opt/anaconda3/envs/fina/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/klemanroy/opt/anaconda3/envs/fina/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/klemanroy/opt/anaconda3/envs/fina/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [7]:
df_loantape_anly["flgtj_bin"] = (
    df_loantape_anly["flgtj"]
        .astype("string")
        .str.strip()
        .notna()
        & (df_loantape_anly["flgtj"].astype("string").str.strip() != "")
).astype(int)

summary = (
    df_loantape_anly
    .groupby(["grdtd", "flgtj_bin"], dropna=False)["account"]
    .nunique()
    .reset_index(name="account_cnt")
)

summary



,grdtd,flgtj_bin,account_cnt
0,N,0,31
1,N,1,21
2,Y,0,160
3,Y,1,68
4,NaN,0,507
5,NaN,1,96


In [8]:
summary = (
    df_loantape_anly
    .groupby(["grdtd"], dropna=False)["account"]
    .nunique()
    .reset_index(name="account_cnt")
)

summary

,grdtd,account_cnt
0,N,52
1,Y,228
2,NaN,603
